In [50]:
import json
import pymongo
from pymongo import MongoClient
from pymongo.errors import OperationFailure, ConnectionFailure

Temos um banco MongoDB e queremos nos conectar a ele e testar a conexão.
Monitorar o status da conexão com banco e receber notificações caso a conexão tenha caído
Pode ser muito útil em sistemas cujo banco não possui notificações implementadas. 

In [66]:
# Primeiro carregamos o secret com as credenciais.
# As credenciais podem ser algo assim: mongodb_uri = "mongodb://localhost:27017"

with open('secret.json', 'r') as js:
          client_json = json.load(js)


In [67]:
# Agora instanciamos o Client e chamamos o database que queremos testar. 
client = pymongo.MongoClient(client_json['client'])
db = client.test # select the database


In [70]:
# Agora vamos buscar a infomração do servidor.
# Ela contem um json com essas informações por exemplo:
# Queremos saber se o banco está ok, então buscamos o parâmetro ok.
# Caso ele seja 1, o banco está ok, caso contrário, não. 

"""{'version': '5.0.15',
 'gitVersion': '935639beed3d0c19c2551c93854b831107c0b118',
 'modules': ['enterprise'],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [5, 0, 15, 0],
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 'storageEngines': ['devnull',
  'ephemeralForTest',
  'inMemory',
  'queryable_wt',
  'wiredTiger'],
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1680743043, 1),
  'signature': {'hash': b'Se\x81\x1ca{\x19O\x16=o\xa9\xe0A>\xe4-\x9eg\xcd',
   'keyId': 7155114180230512642}},
 'operationTime': Timestamp(1680743043, 1)}"""

server_info = client.server_info()

In [71]:
server_info

{'version': '5.0.15',
 'gitVersion': '935639beed3d0c19c2551c93854b831107c0b118',
 'modules': ['enterprise'],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [5, 0, 15, 0],
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 'storageEngines': ['devnull',
  'ephemeralForTest',
  'inMemory',
  'queryable_wt',
  'wiredTiger'],
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1680743123, 1),
  'signature': {'hash': b'\x80q\xdf\\\xe6\x97\xe2\x11D&\xc5\xbf\n2wY\xddA\x8d\x13',
   'keyId': 7155114180230512642}},
 'operationTime': Timestamp(1680743123, 1)}

Nossa notificação será enviada, caso o banco não esteja ok, vamos usar 
import smtplib para isso. 


In [74]:
import smtplib
sender_email = "dsericaferreira@gmail.com"

receiver_email = "dsericaferreira@gmail.com"


In [76]:
if server_info['ok'] != 1:
    subject = "MongoDB status: ERROR"
    body = "Temos um problema com a conexão do banco."
else:
    subject = "MongoDB status: Ok!"
    body = "Conexão Ok!"

In [77]:
with smtplib.SMTP("localhost") as server:
    server.sendmail(sender_email, receiver_email, f"Subject: {subject}\n\n{body}")

ConnectionRefusedError: [Errno 61] Connection refused